In [ ]:
# --- Install dependencies (Colab) ---
!pip install -q torch torchvision torchaudio scikit-learn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.0 MB/s eta 0:00:00


In [ ]:
# --- Imports ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from google.colab import drive
from sklearn.metrics import mean_squared_error
import numpy as np
import math
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
# --- Mount Google Drive ---
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --- Constants for screen setup ---
SCREEN_WIDTH_PX = 800
SCREEN_HEIGHT_PX = 600
SCREEN_WIDTH_MM = 518
SCREEN_HEIGHT_MM = 324
VIEWING_DISTANCE_MM = 680

In [ ]:
# --- Positional Encoding ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1200):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

# --- Patch Embedding via Conv ---
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=1, emb_size=256):
        super().__init__()
        self.proj = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, stride=2, padding=1),
            nn.GELU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.GELU(),
            nn.Conv2d(64, emb_size, kernel_size=3, stride=2, padding=1),
            nn.GELU()
        )

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.proj(x)
        B, E, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        return x

# --- Transformer Encoder Block ---
class TransformerBlock(nn.Module):
    def __init__(self, emb_size=256, heads=4, ff_dim=512, dropout=0.2):
        super().__init__()
        self.attn = nn.MultiheadAttention(emb_size, heads, dropout=dropout, batch_first=True)
        self.ff = nn.Sequential(
            nn.Linear(emb_size, ff_dim),
            nn.GELU(),
            nn.Linear(ff_dim, emb_size)
        )
        self.norm1 = nn.LayerNorm(emb_size, eps=1e-5)
        self.norm2 = nn.LayerNorm(emb_size, eps=1e-5)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        attn_out, _ = self.attn(x, x, x)
        x = self.norm1(x + self.drop(attn_out))
        ff_out = self.ff(x)
        x = self.norm2(x + self.drop(ff_out))
        return x

# --- Full Model with Attention Pooling ---
class EEGTransformerRegressor(nn.Module):
    def __init__(self, emb_size=256, num_layers=4):
        super().__init__()
        self.patch_embed = PatchEmbedding(1, emb_size)
        self.pos_encoder = PositionalEncoding(emb_size)
        self.transformer = nn.Sequential(*[
            TransformerBlock(emb_size) for _ in range(num_layers)
        ])
        self.attn_pool = nn.MultiheadAttention(emb_size, num_heads=4, batch_first=True)
        self.regressor = nn.Sequential(
            nn.LayerNorm(emb_size, eps=1e-5),
            nn.Linear(emb_size, 64),
            nn.GELU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.patch_embed(x)
        x = self.pos_encoder(x)
        x = self.transformer(x)
        pooled, _ = self.attn_pool(x, x, x)
        return self.regressor(pooled[:, 0])

# --- Dataset ---
class EEGNPZDataset(Dataset):
    def __init__(self, file_paths, file_fraction=0.75):
        self.samples = []
        for path in file_paths:
            data = np.load(path)
            eeg = data['EEG']
            labels = data['labels'][:, -2:]
            n = int(len(eeg) * file_fraction)
            for x, y in zip(eeg[:n], labels[:n]):
                if not np.isnan(x).any() and not np.isnan(y).any():
                    deg_x, deg_y = pixels_to_degrees(y[0], y[1])
                    if abs(deg_x) < 30 and abs(deg_y) < 30:
                        x = (x - np.mean(x)) / (np.std(x) + 1e-6)
                        self.samples.append((x, y))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y = self.samples[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [ ]:
def pixels_to_degrees(x_px, y_px):
    x_mm = (x_px - SCREEN_WIDTH_PX / 2) * (SCREEN_WIDTH_MM / SCREEN_WIDTH_PX)
    y_mm = (y_px - SCREEN_HEIGHT_PX / 2) * (SCREEN_HEIGHT_MM / SCREEN_HEIGHT_PX)
    angle_x = np.degrees(np.arctan2(x_mm, VIEWING_DISTANCE_MM))
    angle_y = np.degrees(np.arctan2(y_mm, VIEWING_DISTANCE_MM))
    return angle_x, angle_y

# --- RMSE in degrees of visual angle ---
def rmse_degrees(preds, targets):
    preds_deg = np.array([pixels_to_degrees(x, y) for x, y in preds])
    targets_deg = np.array([pixels_to_degrees(x, y) for x, y in targets])
    return np.sqrt(np.mean((preds_deg - targets_deg) ** 2))
def pixels_to_mm(x_px, y_px):
    x_mm = (x_px - SCREEN_WIDTH_PX / 2) * (SCREEN_WIDTH_MM / SCREEN_WIDTH_PX)
    y_mm = (y_px - SCREEN_HEIGHT_PX / 2) * (SCREEN_HEIGHT_MM / SCREEN_HEIGHT_PX)
    return x_mm, y_mm
def rmse_mm(preds, targets):
    preds_mm = np.array([pixels_to_mm(x, y) for x, y in preds])
    targets_mm = np.array([pixels_to_mm(x, y) for x, y in targets])
    return np.sqrt(np.mean((preds_mm - targets_mm) ** 2))

In [ ]:
# --- Training Loop ---
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, save_dir, patience=5):
    scaler = torch.amp.GradScaler('cuda')
    best_rmse = float('inf')
    train_losses = []
    val_rmses = []
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
            inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)

            with torch.amp.autocast(device_type='cuda'):
                outputs = model(inputs)

            if torch.isnan(outputs).any():
                print(f"❌ NaNs detected in outputs at batch {batch_idx}")
                del inputs, targets, outputs
                torch.cuda.empty_cache()
                continue

            loss = criterion(outputs, targets)

            if not torch.isfinite(loss).all():
                print(f"⚠️ Skipping batch {batch_idx} with non-finite loss: {loss.item()}")
                del inputs, targets, outputs, loss
                torch.cuda.empty_cache()
                continue

            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * inputs.size(0)

            del inputs, targets, outputs, loss
            torch.cuda.empty_cache()

        avg_train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(avg_train_loss)

        model.eval()
        preds, gts = [], []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                preds.append(outputs.cpu().numpy())
                gts.append(targets.cpu().numpy())

        preds = np.concatenate(preds)
        gts = np.concatenate(gts)
        rmse = np.sqrt(mean_squared_error(gts, preds))
        val_rmses.append(rmse)
        rmse_deg = rmse_degrees(preds, gts)
        rmse_mm_val = rmse_mm(preds, gts)

        scheduler.step()

        if rmse < best_rmse:
            best_rmse = rmse
            patience_counter = 0
            torch.save(model.state_dict(), os.path.join(save_dir, "best_model.pt"))
            print("✅ Saved new best model.")
        else:
            patience_counter += 1
            print(f"⏳ No improvement. Patience: {patience_counter}/{patience}")
            if patience_counter >= patience:
                print("🛑 Early stopping triggered.")
                break

        print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {avg_train_loss:.4f} | Val RMSE: {rmse:.2f}px | {rmse_deg:.2f}° | {rmse_mm_val:.2f}mm")

    plt.figure()
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_rmses, label='Validation RMSE (px)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss / RMSE')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.savefig(os.path.join(save_dir, 'loss_plot.png'))
    plt.close()

In [ ]:
data_dir = "/content/drive/MyDrive/data"
files = [
    "Direction_task_with_dots_synchronised_max.npz",
    "Direction_task_with_dots_synchronised_min.npz",
    "Direction_task_with_processing_speed_synchronised_max.npz",
    "Direction_task_with_processing_speed_synchronised_min.npz",
]
file_paths = [os.path.join(data_dir, f) for f in files]

dataset = EEGNPZDataset(file_paths)
train_size = int(0.8 * len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=True)

In [10]:
# --- Execution ---
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = EEGTransformerRegressor().to(device)

    criterion = nn.SmoothL1Loss(beta=10.0)  # beta controls transition to L1
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5)

    save_dir = "/content/drive/MyDrive/cnn_transformer/outliers/Try1"
    os.makedirs(save_dir, exist_ok=True)

    train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, device=device, save_dir=save_dir, patience=5)

Epoch 1: 100%|██████████| 925/925 [05:49<00:00,  2.64it/s]


✅ Saved new best model.
Epoch [1/100] | Train Loss: 89.4318 | Val RMSE: 131.75px | 7.03° | 85.30mm


Epoch 2: 100%|██████████| 925/925 [05:49<00:00,  2.65it/s]


✅ Saved new best model.
Epoch [2/100] | Train Loss: 68.9017 | Val RMSE: 126.83px | 6.76° | 82.12mm


Epoch 3: 100%|██████████| 925/925 [05:49<00:00,  2.65it/s]


...

⏳ No improvement. Patience: 3/5
Epoch [45/100] | Train Loss: 20.2004 | Val RMSE: 71.77px | 3.82° | 46.47mm


Epoch 46: 100%|██████████| 925/925 [05:49<00:00,  2.65it/s]


⏳ No improvement. Patience: 4/5
Epoch [46/100] | Train Loss: 23.0403 | Val RMSE: 73.38px | 3.91° | 47.51mm


Epoch 47: 100%|██████████| 925/925 [05:49<00:00,  2.65it/s]


⏳ No improvement. Patience: 5/5
🛑 Early stopping triggered.
